In [7]:
using Latexify
using Distributions
using DataFrames
using CairoMakie

# Parameters
k_vals = [2,3,5,10, 100, 1000, 1500]  # Number of categories
count_vals = 10 .^ range(0, stop=15, length=100000)  # Total counts (c) on a logarithmic scale
# Initialize DataFrame to store results
results = DataFrame(k=Int[], case=String[],  threshold=Float64[], rel_interval=Union{Float64, Missing}[],c=Union{Int, Missing}[], mean=Union{Float64, Missing}[], lower=Union{Float64, Missing}[], upper=Union{Float64, Missing}[])
case_names = ["alpha_i = c / k + 1", "alpha_i = c / 2 + 1", "alpha_i = c / (2k) + 1"]

# Analyze each configuration
for k in k_vals
    cases = [(case_names[1], c -> c / k + 1), (case_names[2], c -> c / 2 + 1),  (case_names[3], c -> c / (2 * k) + 1)]
    for (label, alpha_func) in cases
        for threshold in [0.3, 0.1, 0.05]
            found = false
            for c in count_vals
                c = round(Int, c)
                alpha0 = k + c  # Total concentration
                alpha_i = alpha_func(c)  # Calculate alpha_i based on the case
                beta_i = alpha0 - alpha_i

                # Marginal Beta distribution
                beta_dist = Beta(alpha_i, beta_i)

                # Compute mean and 95% credible interval
                mean_val = alpha_i / alpha0
                lower = quantile(beta_dist, 0.025)
                upper = quantile(beta_dist, 0.975)
                relative_interval = (upper - lower) / mean_val

                if relative_interval < threshold
                    push!(results, (k, label, threshold, relative_interval, c, mean_val, lower, upper))
                    found = true
                    break
                end
            end
            if !found
                push!(results, (k, label, threshold, missing, missing, missing, missing, missing))
            end
        end
    end
end



In [8]:

# Display the results
sorted = sort(results, [:case, :k, :rel_interval], rev=[true, false, true])[!, [:case,:k,:c, :mean, :lower, :upper, :rel_interval, :threshold ]]

# Add a column c / k
sorted[!, "c_div_k"] = [ismissing(row.c) ? missing : row.c / row.k for row in eachrow(sorted)]
println(sorted)

63×9 DataFrame
 Row │ case                    k      c         mean         lower        upper        rel_interval  threshold  c_div_k      
     │ String                  Int64  Int64?    Float64?     Float64?     Float64?     Float64?      Float64    Float64      
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ alpha_i = c / k + 1         2       168  0.5          0.425153     0.574847        0.299388        0.3      84.0
   2 │ alpha_i = c / k + 1         2      1534  0.5          0.475007     0.524993        0.0999727       0.1     767.0
   3 │ alpha_i = c / k + 1         2      6144  0.5          0.487501     0.512499        0.0499956       0.05   3072.0
   4 │ alpha_i = c / k + 1         3       337  0.333333     0.284266     0.384258        0.299978        0.3     112.333
   5 │ alpha_i = c / k + 1         3      3069  0.333333     0.31677      0.350102        0.0999944       0.1    1023.0
   6 

In [9]:
using CSV

# Save the DataFrame to a CSV file
CSV.write("credible_intervals_dataframe.csv", sorted)

# You can also use PrettyTables for formatted text file output
using PrettyTables
open("credible_intervals.txt", "w") do io
    pretty_table(io, sorted)
end


In [16]:
function create_latex_table_with_thresholds(data, case, thresholds)
    table = ""
    table *= "\\begin{tabular}{|r|" * join(["r" for _ in thresholds], "|") * "|}\n\\hline\n"
    table *= "\$k\$ & " * join(["\$" * string(thr) * "\$" for thr in thresholds], " & ") * " \\\\\n\\hline\n"
    for k in unique(data.k)
        row_values = [filter(row -> row.k == k && row.case == case && row.threshold == thr, eachrow(data)) for thr in thresholds]
        entries = [isempty(rows) ? "-" : isnothing(rows[1].c) ? "-" : string(rows[1].c) for rows in row_values]
        table *= "\$$k\$ & \$" * join(entries, "\$ & \$") * "\$ \\\\\n"
    end
    table *= "\\hline\n\\end{tabular}\n"
    return table
end


# Example usage:
thresholds = [0.3, 0.1, 0.05]
case = case_names[1]
latex_table = create_latex_table_with_thresholds(
    sorted,
    case,
    thresholds
)

println(latex_table)


\begin{tabular}{|r|r|r|r|}
\hline
$k$ & $0.3$ & $0.1$ & $0.05$ \\
\hline
$2$ & $168$ & $1534$ & $6144$ \\
$3$ & $169$ & $1535$ & $6144$ \\
$5$ & $171$ & $1537$ & $6146$ \\
$10$ & $175$ & $1542$ & $6152$ \\
$100$ & $220$ & $1621$ & $6238$ \\
$1000$ & $300$ & $2039$ & $6919$ \\
$1500$ & $310$ & $2163$ & $7202$ \\
\hline
\end{tabular}



In [17]:

case = case_names[2]
latex_table = create_latex_table_with_thresholds(
    sorted,
    case,
    thresholds
)

println(latex_table)

\begin{tabular}{|r|r|r|r|}
\hline
$k$ & $0.3$ & $0.1$ & $0.05$ \\
\hline
$2$ & $168$ & $1534$ & $6144$ \\
$3$ & $169$ & $1535$ & $6144$ \\
$5$ & $171$ & $1537$ & $6146$ \\
$10$ & $175$ & $1542$ & $6152$ \\
$100$ & $220$ & $1621$ & $6238$ \\
$1000$ & $300$ & $2039$ & $6919$ \\
$1500$ & $310$ & $2163$ & $7202$ \\
\hline
\end{tabular}



In [18]:

case = case_names[3]
latex_table = create_latex_table_with_thresholds(
    sorted,
    case,
    thresholds
)

println(latex_table)

\begin{tabular}{|r|r|r|r|}
\hline
$k$ & $0.3$ & $0.1$ & $0.05$ \\
\hline
$2$ & $506$ & $4605$ & $18433$ \\
$3$ & $846$ & $7677$ & $30732$ \\
$5$ & $1524$ & $13819$ & $55322$ \\
$10$ & $3219$ & $29171$ & $116775$ \\
$100$ & $33748$ & $305566$ & $1223238$ \\
$1000$ & $339043$ & $3069834$ & $12284879$ \\
$1500$ & $508577$ & $4604862$ & $18434130$ \\
\hline
\end{tabular}

